In [6]:
# Author: Calli 
# Date Created: Jan 2020
# Purpose: Coursera SQL Final Project

# Importing Libraries --------------------
import pandas as pd
import os 
import numpy as np
import ibm_db_sa,ibm_db_sa
import sqlalchemy
from sqlalchemy import *
import os.path
import json
import ibm_db

In [7]:
# Loading Data from Cloud -------------

#Loading JSON Token:
with open('token/ibm_token.json') as json_data:
   token = json.load(json_data)

#Identify the Credentials:

dsn_hostname = token['hostname']
dsn_uid = token['username']        
dsn_pwd = token['password']

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = token['db']            
dsn_port = str(token['port'])                
dsn_protocol = 'TCPIP' 

dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

print(dsn)

#Create Database Connection:

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg())

#Retrieve Metadata for the Database Server
server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

#Retrieve Metadata for the Database Client / Driver
client = ibm_db.client_info(conn)

print ("DRIVER_NAME:          ", client.DRIVER_NAME) 
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)
    
#Close the Connection
#ibm_db.close(conn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=hjr21682;PWD=m+x8dwxcv5c66cp1;
Connected to database:  BLUDB as user:  hjr21682 on host:  dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net
DBMS_NAME:  DB2/LINUXX8664
DBMS_VER:   11.01.0004
DB_NAME:    BLUDB
DRIVER_NAME:           libdb2.a
DRIVER_VER:            10.05.0005
DATA_SOURCE_NAME:      BLUDB
DRIVER_ODBC_VER:       03.51
ODBC_VER:              03.01.0000
ODBC_SQL_CONFORMANCE:  EXTENDED
APPL_CODEPAGE:         1208
CONN_CODEPAGE:         1208


In [8]:
# Accessing Databases with SQl Magic ------------------

%load_ext sql
%sql ibm_db_sa://hjr21682:m+x8dwxcv5c66cp1@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB

'Connected: hjr21682@BLUDB'

In [9]:
# Saving SQL Table locally 

df_raw = %sql select * from crimechicago

 * ibm_db_sa://hjr21682:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


In [10]:
#Converting SQL table to pandas dataframe:

#Method 1: Using pandas
df = pd.DataFrame(df_raw)

#Method 2: Using SQL Magic
#df = tablename.DataFrame()
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,3512276,HK587712,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,FALSE,FALSE,911,9,14.0,58.0,6,1155838.0,1873050.0,2004,41.80744050,-87.70395585,"(41.8074405, -87.703955849)"
1,3406613,HK456306,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,FALSE,FALSE,1112,11,27.0,23.0,6,1152206.0,1906127.0,2004,41.89827996,-87.71640551,"(41.898279962, -87.716405505)"
2,8002131,HT233595,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,FALSE,FALSE,221,2,3.0,38.0,6,1177436.0,1876313.0,2011,41.81593313,-87.62464213,"(41.815933131, -87.624642127)"
3,7903289,HT133522,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,FALSE,FALSE,423,4,7.0,46.0,6,1194622.0,1850125.0,2010,41.74366532,-87.56246276,"(41.743665322, -87.562462756)"
4,10402076,HZ138551,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,FALSE,FALSE,831,8,15.0,66.0,6,1155240.0,1860661.0,2016,41.77345530,-87.70648047,"(41.773455295, -87.706480471)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,10453948,HZ192829,028XX E 79TH ST,1055,HUMAN TRAFFICKING,INVOLUNTARY SERVITUDE,APARTMENT,FALSE,FALSE,422,4,7.0,46.0,26,1196679.0,1853139.0,2016,41.75188515,-87.55482600,"(41.751885152, -87.554825997)"
529,10397129,HZ133234,006XX W HARRISON ST,5114,NON - CRIMINAL,FOID - REVOCATION,STREET,FALSE,FALSE,124,1,2.0,28.0,26,1172257.0,1897564.0,2016,41.87436328,-87.64301304,"(41.874363279, -87.643013039)"
530,3269495,HJ747227,012XX W 81ST ST,510,RITUALISM,AGG RIT MUT: HANDS/FIST/FEET SERIOUS INJURY,OTHER,FALSE,FALSE,612,6,21.0,71.0,04B,1169648.0,1851076.0,2003,41.74685249,-87.65394139,"(41.746852486, -87.653941385)"
531,10840565,JA143710,009XX N KARLOV AVE,5073,NON-CRIMINAL (SUBJECT SPECIFIED),NOTIFICATION OF CIVIL NO CONTACT ORDER,RESIDENCE,FALSE,TRUE,1111,11,37.0,23.0,26,1148881.0,1905963.0,2017,41.89789489,-87.72862232,"(41.89789489, -87.728622316)"


In [11]:
# Problem 1 : Find the total number of crimes recorded in the CRIME table

#Method 1: Querying data from SQL DB
%sql select count(*) from crimechicago

# Method 2: Looking at the Object
#df.shape[0]

 * ibm_db_sa://hjr21682:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


1
533


In [12]:
# Problem 2: Retrieve first 10 rows from the CRIME table

%sql select * from crimechicago LIMIT 10

 * ibm_db_sa://hjr21682:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


id,case_number,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area_number,fbicode,x_coordinate,y_coordinate,YEAR,latitude,longitude,location
3512276,HK587712,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,FALSE,FALSE,911,9,14,58,6,1155838,1873050,2004,41.80744050,-87.70395585,"(41.8074405, -87.703955849)"
3406613,HK456306,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,FALSE,FALSE,1112,11,27,23,6,1152206,1906127,2004,41.89827996,-87.71640551,"(41.898279962, -87.716405505)"
8002131,HT233595,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,FALSE,FALSE,221,2,3,38,6,1177436,1876313,2011,41.81593313,-87.62464213,"(41.815933131, -87.624642127)"
7903289,HT133522,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,FALSE,FALSE,423,4,7,46,6,1194622,1850125,2010,41.74366532,-87.56246276,"(41.743665322, -87.562462756)"
10402076,HZ138551,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,FALSE,FALSE,831,8,15,66,6,1155240,1860661,2016,41.77345530,-87.70648047,"(41.773455295, -87.706480471)"
7732712,HS540106,006XX W CHICAGO AVE,810,THEFT,OVER $500,PARKING LOT/GARAGE(NON.RESID.),FALSE,FALSE,1323,12,27,24,6,1171668,1905607,2010,41.89644677,-87.64493868,"(41.896446772, -87.644938678)"
10769475,HZ534771,050XX N KEDZIE AVE,810,THEFT,OVER $500,STREET,FALSE,FALSE,1713,17,33,14,6,1154133,1933314,2016,41.97284491,-87.70860008,"(41.972844913, -87.708600079)"
4494340,HL793243,005XX E PERSHING RD,860,THEFT,RETAIL THEFT,GROCERY FOOD STORE,TRUE,FALSE,213,2,3,38,6,1180448,1879234,2005,41.82387989,-87.61350386,"(41.823879885, -87.613503857)"
3778925,HL149610,100XX S WASHTENAW AVE,810,THEFT,OVER $500,STREET,FALSE,FALSE,2211,22,19,72,6,1160129,1838040,2005,41.71128051,-87.68917910,"(41.711280513, -87.689179097)"
3324217,HK361551,033XX W BELMONT AVE,820,THEFT,$500 AND UNDER,SMALL RETAIL STORE,FALSE,FALSE,1733,17,35,21,6,1153590,1921084,2004,41.93929582,-87.71092344,"(41.939295821, -87.710923442)"


In [13]:
# Problem 3: How many crimes involve an arrest?

%sql select count(*) from crimechicago where arrest = true 

 * ibm_db_sa://hjr21682:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


1
163


In [14]:
# Problem 4: Which unique types of crimes have been recorded at GAS STATION locations?

%sql select DISTINCT(primary_type) from crimechicago where location_description = 'GAS STATION'

 * ibm_db_sa://hjr21682:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


primary_type
CRIMINAL TRESPASS
NARCOTICS
ROBBERY
THEFT


In [15]:
# Problem 5: In the CENsUS_DATA table list all Community Areas whose names start with the letter ‘B’.

%sql select COMMUNITY_AREA_NAME from socioecochicago where COMMUNITY_AREA_NAME like 'B%'

 * ibm_db_sa://hjr21682:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name
Belmont Cragin
Burnside
Brighton Park
Bridgeport
Beverly


In [16]:
# Problem 6: Which schools in Community Areas 10 to 15 are healthy school certified?

%sql select name_of_school from publicschoolschicago \
where COMMUNITY_AREA_NUMBER between 10 and 15 \
IN (select HEALTHY_SCHOOL_CERTIFIED from publicschoolschicago where HEALTHY_SCHOOL_CERTIFIED = 'Yes')

 * ibm_db_sa://hjr21682:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school
Albany Park Multicultural Academy
Alessandro Volta Elementary School
Daniel C Beard Elementary School
Edgebrook Elementary School
Edison Park Elementary School
Edwin G Foreman High School
Ernst Prussing Elementary School
Friedrich W von Steuben Metropolitan Science High School
Hannah G Solomon Elementary School
Helge A Haugan Elementary School


In [17]:
# Problem 7: What is the average school Safety Score?

%sql select AVG(SAFETY_SCORE) from publicschoolschicago

 * ibm_db_sa://hjr21682:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


1
49.504873


In [18]:
# Problem 8: List the top 5 Community Areas by average College Enrollment [number of students]

%sql select community_area_name, college_enrollment from publicschoolschicago order by college_enrollment desc LIMIT 5;

 * ibm_db_sa://hjr21682:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,college_enrollment
NORTH CENTER,4368
ARCHER HEIGHTS,3320
NORWOOD PARK,2922
BRIGHTON PARK,2883
IRVING PARK,2366


In [19]:
# Problem 9: Use a sub-query to determine which Community Area has the least value for school Safety Score?

%sql select community_area_name,SAFETY_SCORE  from publicschoolschicago where SAFETY_SCORE = \
(select min(SAFETY_SCORE) from publicschoolschicago);

 * ibm_db_sa://hjr21682:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,safety_score
WASHINGTON PARK,1


In [20]:
# Problem 10: [Without using an explicit JOIN operator] Find the Per Capita Income of the Community Area which has a school Safety Score of 1.

%sql select COMMUNITY_AREA_NAME, PER_CAPITA_INCOME from socioecochicago where \
UCASE(COMMUNITY_AREA_NAME) in (select COMMUNITY_AREA_NAME from publicschoolschicago where SAFETY_SCORE = 1);

 * ibm_db_sa://hjr21682:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,per_capita_income
Washington Park,13785
